In [1]:
import pandas as pd
from source.project_manager import (
    load_slurm_metadata,
    delete_slurm_data,
    check_slurm_data_exists,
)

In [2]:
project_metadata = load_slurm_metadata()

In [4]:
project_metadata["paths"].apply(check_slurm_data_exists)

0    False
1    False
2    False
3    False
4    False
5    False
0    False
1    False
2    False
3    False
4    False
5    False
0    False
1    False
2    False
3    False
4    False
5    False
0    False
1    False
2    False
3    False
4    False
5    False
Name: paths, dtype: bool

In [4]:
project_data

NameError: name 'project_data' is not defined